In [377]:
import camelot
import numpy as np
import pandas as pd
import json
from stuff import output_directory, tablestocellinfo, file, filename, cleanuptables
import os
import sys

In [387]:
def is_first_page(df):
    for x in df.iloc[:,0].values:
        if 'Hæf' in x:
            return True
    return False

def get_first_date_cell(df):
    import re
    for row_idx,row in df.iterrows():
        for col_idx, cell in enumerate(row):
            # print(f'row {row_idx}, col {col_idx}, cell: {cell}')
            if re.match('[0-9][0-9]\.[0-9][0-9]', cell):
                return col_idx,row_idx

def get_num_pages(tables):
    counts = []
    last_first_page = 0
    for idx,table in enumerate(tables):
        if is_first_page(table.df):
            if idx > 0:
                counts.append(idx - last_first_page)
                last_first_page = idx
    if len(set(counts)) == 1:
        return counts[0]
    else:
        return counts

In [380]:
tables = camelot.read_pdf(file,pages='1-end')
cellinfo = tablestocellinfo(tables)

In [141]:
for table in tables:
    print(is_first_page(table.df),get_first_date_cell(table.df))

True (0, 0)
False (2, 0)
False (1, 0)
True (0, 0)
False (2, 0)
False (1, 0)
True (0, 0)
False (2, 0)
False (1, 0)


In [386]:
get_num_pages(tables)

3

In [219]:
pages = []

for idx,table in enumerate(tables):
    if is_first_page(table.df):

        temp = table.df.iloc[1:,1:]
        temp.iloc[0,0] = 'Starfsmaður'
        pages.append(temp)
        # i = i + 1
        # print(table.df.iloc[2:,1:].head())
    else:
        first_page = pages[len(pages)-1]
        if not get_first_date_cell(first_page):
            x,y = get_first_date_cell(table.df)
            first_page.iloc[0,1:] = table.df.iloc[y,x:]
            x,y = get_first_date_cell(tables[1].df)
            # print(x,y)
            current_page = table.df.iloc[y+1:,x-1:]
            first_page = pd.concat([first_page,current_page],ignore_index=True)
            # print(first_page)
            # first_page.to_csv(os.path.join(output_directory,str(idx) + 'table.csv'))
            




In [193]:
x,y = get_first_date_cell(tables[1].df)
print(x,y)
tables[1].df.iloc[y+1:,x-1:]

2 0


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,Kristján Ingi Rúnar...,,,,,,,,,,16:00-23:00,16:00-23:00,,,
2,Kristján Kristjánsson,,,,,,,08:00-16:00,,16:00-23:00,12:00-20:00,12:00-20:00,,,
3,Lena María Brynjar...,,,,,,16:00-23:00,16:00-23:00,,,16:00-22:00,16:00-22:00,,16:00-23:00,
4,Óðinn Dagur Bjarn...,16:00-22:00,,,,08:00-16:00,16:00-22:00,,08:00-16:00,,,,08:00-16:00,,16:00-22:00
5,Ólafur Árni Jónsson,ORLOF,ORLOF,08:00-16:00,08:00-16:00,,08:00-16:00,14:00-22:00,08:00-16:00,,,,08:00-16:00,08:00-16:00,14:00-22:00
6,Ragnar Ingi Karlsson,,,12:00-20:00,12:00-20:00,,,,,,,,,,
7,Ragnheiður Lindud...,,,09:00-16:00,09:00-16:00,15:00-23:00,,,,,,,,,
8,Rakel Ketilsdóttir,23:00-09:00,,,,,15:00-23:00,12:00-22:00,,14:00-22:00,09:00-16:00,09:00-16:00,,08:00-16:00,12:00-22:00
9,Róbert Steinar Hjál...,14:00-22:00,08:00-16:00,,,08:00-16:00,08:00-16:00,12:00-20:00,16:00-22:00,,12:00-20:00,12:00-20:00,,08:00-16:00,
10,Rósa María Friðrik...,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF


In [376]:
for offset in range(0,9,3):
    pages = []

    first_page = tables[0+offset].df.iloc[1:,1:].copy()
    first_page.iloc[0,0] = 'Starfsmaður'


    second_page = tables[1+offset].df.copy()

    third_page = tables[2+offset].df.copy()

    first_page.index -= 1
    first_page.columns -= 1
    # second_page.index -= 1
    # third_page.index -= 1
    # for page in [first_page,second_page,third_page]:
    #     page.reset_index()

    x,y = get_first_date_cell(second_page)
    first_page.iloc[0,1:] = second_page.iloc[y,x:]

    second_page = second_page.iloc[y+1:,x-1:]
    second_page.index -= 1 
    second_page.columns -= 1

    x,y = get_first_date_cell(third_page)

    # third_page.to_csv(os.path.join(output_directory,'thirdpage.csv'))
    third_page = third_page.iloc[y+1:,x-1:]
    # third_page.to_csv(os.path.join(output_directory,'thirdpage2.csv'))

    joined = pd.concat([first_page,second_page,third_page],ignore_index = True)
    joined.iloc[:,:].to_csv(os.path.join(output_directory,'out' + str(offset) + '.csv'))

    # second_page.index,third_page.index = first_page.index,first_page.index



    # for idx,table in enumerate(tables):
    #         first_page = pages[len(pages)-1]
    #         if not get_first_date_cell(first_page):
    #             x,y = get_first_date_cell(table.df)
    #             first_page.iloc[0,1:] = table.df.iloc[y,x:]
    #             x,y = get_first_date_cell(tables[1].df)
    #             # print(x,y)
    #             current_page = table.df.iloc[y+1:,x-1:]
    #             first_page = pd.concat([first_page,current_page],ignore_index=True)
    #             # print(first_page)
    #             # first_page.to_csv(os.path.join(output_directory,str(idx) + 'table.csv'))

In [140]:
for table in tables:
    print(table.shape)

(27, 16)
(29, 16)
(3, 15)
(28, 16)
(29, 16)
(2, 15)
(28, 16)
(29, 16)
(2, 15)


In [131]:
x,y = get_first_date_cell(tables[5].df)
print(x,y)
print(tables[5].df.iloc[y,x])
tables[5].df

1 0
25.08
fim


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,t\nStarfsmaður,25.08\nfim,26.08\nfös,27.08\nlau,28.08\nsun,29.08\nmán,30.08\nþri,31.08\nmið,01.09\nfim,02.09\nfös,03.09\nlau,04.09\nsun,05.09\nmán,06.09\nþri,07.09\nmið
1,övantar KVK 2,,,,,,,23:00-09:00,,23:00-09:00,23:00-09:00,23:00-09:00,,,


In [132]:
temp = tables[0].df.iloc[:,:]
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,11.08\n12.08\n13.08\n14.08\n15.08\n16.08\n17.0...,,,,,,,,,,,,,,,
1,f\nHæfniþáttur : VEL.stuðningur,,,,,,,,,,,,,,,
2,,Agnes Rut Ásgeirs...,,16:00-23:00,08:00-16:00,08:00-16:00,,,,15:00-23:00,,,,,,
3,,Arndís prufa,,15:00-23:00,16:00-23:00,16:00-23:00,,,,12:00-23:00,,,,16:00-23:00,,
4,,Aron Freyr Kristján...,,,,,08:00-16:00,09:00-20:00,,09:00-20:00,,08:00-16:00,08:00-16:00,23:00-09:00,,16:00-23:00


In [220]:
pages

[                         1            2            3            4   \
 1               Starfsmaður   11.08\nfim   12.08\nfös   13.08\nlau   
 2      Agnes Rut Ásgeirs...               16:00-23:00  08:00-16:00   
 3              Arndís prufa               15:00-23:00  16:00-23:00   
 4    Aron Freyr Kristján...                                          
 5     Benedikt Pétur Sig...        ORLOF        ORLOF                
 6       Björg Hákonardóttir  16:00-22:00                             
 7        Breki Benediktsson               16:00-22:00  16:00-22:00   
 8          Daníel Pétursson               09:00-20:00  16:00-23:00   
 9     Davíð Arnljótur Ka...  08:00-16:00  08:00-16:00                
 10    Friðberg Reynir Tr...        ORLOF        ORLOF                
 11     Guðbjartur Máni G...               16:00-22:00  08:00-16:00   
 12      Guðni Þór Björnsson        ORLOF                             
 13    Gunnar Kristinn Þó...               23:00-09:00  23:00-09:00   
 14   

In [218]:
pd.concat([tables[0].df.iloc[1:,1:],tables[1].df],ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,Starfsmaður,11.08\nfim,12.08\nfös,13.08\nlau,14.08\nsun,15.08\nmán,16.08\nþri,17.08\nmið,18.08\nfim,19.08\nfös,20.08\nlau,21.08\nsun,22.08\nmán,23.08\nþri,24.08\nmið
1,NaN,Agnes Rut Ásgeirs...,,16:00-23:00,08:00-16:00,08:00-16:00,,,,15:00-23:00,,,,,,
2,NaN,Arndís prufa,,15:00-23:00,16:00-23:00,16:00-23:00,,,,12:00-23:00,,,,16:00-23:00,,
3,NaN,Aron Freyr Kristján...,,,,,08:00-16:00,09:00-20:00,,09:00-20:00,,08:00-16:00,08:00-16:00,23:00-09:00,,16:00-23:00
4,NaN,Benedikt Pétur Sig...,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF,ORLOF,ORLOF,,,ORLOF,,
5,NaN,Björg Hákonardóttir,16:00-22:00,,,,16:00-22:00,,,,,08:00-16:00,08:00-16:00,,15:00-22:00,
6,NaN,Breki Benediktsson,,16:00-22:00,16:00-22:00,16:00-22:00,,,09:00-20:00,,,,,09:00-20:00,,
7,NaN,Daníel Pétursson,,09:00-20:00,16:00-23:00,16:00-23:00,16:00-23:00,,,,,,,,,
8,NaN,Davíð Arnljótur Ka...,08:00-16:00,08:00-16:00,,,09:00-20:00,,08:00-16:00,,16:00-22:00,08:00-16:00,08:00-16:00,,,08:00-16:00
9,NaN,Friðberg Reynir Tr...,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF


In [235]:
first_page.index -= 1
first_page.index

RangeIndex(start=0, stop=26, step=1)

In [368]:
t1 = tables[1].df.copy()
t2 = tables[2].df.copy()

# t1 = t1.iloc[1:,:]
# t1 = t1.drop(t1.columns[[0]],axis=1)
t2 = t2.iloc[1:,:]

# print(get_first_date_cell(t2))
# pd.concat([t1,t2],ignore_index=True)

# t = t1.drop(t1.columns[[0]],axis=1)
# t2

print(len(t1.iloc[0]), len(t2.iloc[0]))

t1.columns -=1


# t2

out = pd.concat(
    [
        t1,
        t2,
    ],
    ignore_index=True
)

16 15


In [371]:
out.drop(out.columns[[0]],axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Starfsmaður,11.08\nfim,12.08\nfös,13.08\nlau,14.08\nsun,15.08\nmán,16.08\nþri,17.08\nmið,18.08\nfim,19.08\nfös,20.08\nlau,21.08\nsun,22.08\nmán,23.08\nþri,24.08\nmið
1,Kristján Ingi Rúnar...,,,,,,,,,,16:00-23:00,16:00-23:00,,,
2,Kristján Kristjánsson,,,,,,,08:00-16:00,,16:00-23:00,12:00-20:00,12:00-20:00,,,
3,Lena María Brynjar...,,,,,,16:00-23:00,16:00-23:00,,,16:00-22:00,16:00-22:00,,16:00-23:00,
4,Óðinn Dagur Bjarn...,16:00-22:00,,,,08:00-16:00,16:00-22:00,,08:00-16:00,,,,08:00-16:00,,16:00-22:00
5,Ólafur Árni Jónsson,ORLOF,ORLOF,08:00-16:00,08:00-16:00,,08:00-16:00,14:00-22:00,08:00-16:00,,,,08:00-16:00,08:00-16:00,14:00-22:00
6,Ragnar Ingi Karlsson,,,12:00-20:00,12:00-20:00,,,,,,,,,,
7,Ragnheiður Lindud...,,,09:00-16:00,09:00-16:00,15:00-23:00,,,,,,,,,
8,Rakel Ketilsdóttir,23:00-09:00,,,,,15:00-23:00,12:00-22:00,,14:00-22:00,09:00-16:00,09:00-16:00,,08:00-16:00,12:00-22:00
9,Róbert Steinar Hjál...,14:00-22:00,08:00-16:00,,,08:00-16:00,08:00-16:00,12:00-20:00,16:00-22:00,,12:00-20:00,12:00-20:00,,08:00-16:00,
